# Toronto Clustering and Segmenting

### Part III: Explore Toronto Neighborhoods

In [1]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M6A,North York,"Lawrence Manor, Lawrence Heights"
2,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
3,M3B,North York,Don Mills
4,M6B,North York,Glencairn


In [2]:
# Delete rows where the Borough is Not assigned

df_tor = df_data_1[df_data_1.Borough != 'Not assigned']
df_tor.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M6A,North York,"Lawrence Manor, Lawrence Heights"
2,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
3,M3B,North York,Don Mills
4,M6B,North York,Glencairn
5,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov..."
6,M3C,North York,Don Mills
7,M6C,York,Humewood-Cedarvale
8,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
10,M6E,York,Caledonia-Fairbanks


In [3]:
# Determine shape of new data set

df_tor.shape

(103, 3)

In [4]:
# insert longitude and latitude data

body = client_3f1deb7951654b268b5b4e4a2843682f.get_object(Bucket='torontoclusteringandsegmentingass-donotdelete-pr-nszeh0x0xrymjq',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# Determine shape of lat/long data set

df_data_2.shape

(103, 3)

In [6]:
# merge dataframes from Toronto location and latitude/longitude and sort by Postal Code

df_results = pd.merge(df_tor, df_data_2, on = 'Postal Code')
df_results.sort_values(by = ['Postal Code'])
df_results


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
2,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
6,M3C,North York,Don Mills,43.725900,-79.340923
7,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
8,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


### Map the City of Toronto and Downtown Toronto using Folium

In [7]:
# Import Python libraries 

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [60]:
# The code was removed by Watson Studio for sharing.

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [12]:
# The code was removed by Watson Studio for sharing.

In [14]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M6G,Downtown Toronto,Christie,43.669542,-79.422564
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
3,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
7,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
8,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [15]:
# The code was removed by Watson Studio for sharing.

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [17]:
# The code was removed by Watson Studio for sharing.

### Explore Queen's Park, Ontario Provincial Government in Downtown Toronto using FourSquare

In [18]:
# The code was removed by Watson Studio for sharing.

In [27]:
# The code was removed by Watson Studio for sharing.

Latitude and longitude values of Queen's Park, Ontario Provincial Government are 43.6623015, -79.3894938.


In [31]:
# The code was removed by Watson Studio for sharing.

### Determine the Top 50 venues in Downtown Toronto within 250 meters

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edeebf598205d001b97222a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.65857210225, 'lng': -79.37781199347744},
   'sw': {'lat': 43.654072097749996, 'lng': -79.38402020652255}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57eda381498ebe0e6ef40972',
       'name': 'UNIQLO ユニクロ',
       'location': {'address': '220 Yonge St',
        'crossStreet': 'at Dundas St W',
        'lat': 43.65591027779457,
        'lng': -79.38064099181345,
        'labeledLatLng

In [36]:
# The code was removed by Watson Studio for sharing.

In [39]:
# The code was removed by Watson Studio for sharing.

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Ed Mirvish Theatre,Theater,43.655102,-79.379768
2,DAVIDsTEA,Tea Room,43.656973,-79.381040
3,Silver Snail Comics,Comic Shop,43.657031,-79.381403
4,Yonge-Dundas Square,Plaza,43.656054,-79.380495
5,CF Toronto Eaton Centre,Shopping Mall,43.654540,-79.380677
6,Barberian's Steak House,Steakhouse,43.657755,-79.382177
7,Crepe Delicious,Fast Food Restaurant,43.654536,-79.380889
8,Burrito Boyz,Burrito Place,43.656265,-79.378343
9,Jazz Bistro,Music Venue,43.655678,-79.379276


In [42]:
# The code was removed by Watson Studio for sharing.

49 venues in Queen's Park/Ontario Provincial Government were returned by Foursquare.


In [59]:
# The code was removed by Watson Studio for sharing.

In [53]:
# The code was removed by Watson Studio for sharing.

Christie
Queen's Park, Ontario Provincial Government
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city


In [46]:
# The code was removed by Watson Studio for sharing.

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,3,3,3,3,3,3
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",2,2,2,2,2,2
Central Bay Street,24,24,24,24,24,24
Christie,6,6,6,6,6,6
Church and Wellesley,35,35,35,35,35,35
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",46,46,46,46,46,46
"Garden District, Ryerson",43,43,43,43,43,43
"Harbourfront East, Union Station, Toronto Islands",38,38,38,38,38,38


There are 19 neighborhoods in Downtown Toronto. The neighborhoods CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island Airport have the least amount of venues, 2. Commerce Court, Victoria Hotel, Toronto Dominion Centre, and Design Exchange neighborhoods have the most venues, 50.

In [50]:
# The code was removed by Watson Studio for sharing.

There are a total of 138 venues in downtown Toronto.

In [49]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Yoga Studio,Adult Boutique,Airport Lounge,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.5,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Christie,0.000000,0.000000,0.0,0.166667,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.028571,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.028571,0.000000,0.028571,0.00000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0,0.040000,0.00000,0.020000,0.00000,0.000000,0.000000,...,0.020000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.0,0.021739,0.00000,0.021739,0.00000,0.000000,0.000000,...,0.043478,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.0,0.000000,0.00000,0.023256,0.00000,0.000000,0.000000,...,0.023256,0.023256,0.000000,0.00000,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.052632,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.0,0.000000,0.02439,0.000000,0.02439,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.02439,0.024390,0.000000,0.048780,0.024390,0.000000,0.000000


List the Top 5 venues in each Downtown Toronto neighborhood.

Berczy Park neighborhood has the highest frequency of venues - Breakfast Spot, Restaurant and Liquor Store.

In [52]:
# The code was removed by Watson Studio for sharing.

----Berczy Park----
                 venue  freq
0       Breakfast Spot  0.33
1           Restaurant  0.33
2         Liquor Store  0.33
3  Moroccan Restaurant  0.00
4               Lounge  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                   venue  freq
0  Performing Arts Venue   0.5
1         Airport Lounge   0.5
2                 Museum   0.0
3                 Lounge   0.0
4      Martial Arts Dojo   0.0


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.29
1                       Café  0.08
2  Middle Eastern Restaurant  0.08
3             Sandwich Place  0.08
4                       Bank  0.04


----Christie----
                  venue  freq
0   American Restaurant  0.17
1         Grocery Store  0.17
2  Gym / Fitness Center  0.17
3           Coffee Shop  0.17
4           Candy Store  0.17


----Church and Wellesley----
          venue  freq
0   Coffee S

List the Top 10 most common venues in each Downtown Toronto neighborhood

In [55]:
# The code was removed by Watson Studio for sharing.

In [58]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Restaurant,Breakfast Spot,Liquor Store,Wings Joint,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Distribution Center
1,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Airport Lounge,Dance Studio,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cupcake Shop
2,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Café,Sandwich Place,Fried Chicken Joint,Spa,Pharmacy,Pizza Place,Deli / Bodega,Restaurant
3,Christie,Gym / Fitness Center,Candy Store,Coffee Shop,American Restaurant,Japanese Restaurant,Grocery Store,Distribution Center,Discount Store,Diner,Dessert Shop
4,Church and Wellesley,Burger Joint,Coffee Shop,Dessert Shop,Wings Joint,Japanese Restaurant,Restaurant,Ramen Restaurant,Polish Restaurant,Park,Mexican Restaurant
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Deli / Bodega,Gastropub,Gym,American Restaurant,Japanese Restaurant,Hotel,Restaurant,Seafood Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Restaurant,Salad Place,Deli / Bodega,Thai Restaurant,Café,Bar,Gym,Bakery,Seafood Restaurant
7,"Garden District, Ryerson",Coffee Shop,Café,Middle Eastern Restaurant,Burger Joint,Greek Restaurant,Bookstore,College Rec Center,Ramen Restaurant,Pub,Plaza
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Pizza Place,Sports Bar,Café,Theater,Plaza,Bubble Tea Shop,Sandwich Place,Salad Place,Gym
9,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Bakery,Noodle House,Organic Grocery,Park,Dessert Shop,Cheese Shop


## Cluster the Various Neighborhoods

### Cluster the neighborhoods into 5 clusters

In [63]:
# The code was removed by Watson Studio for sharing.

array([3, 2, 1, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

In [62]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4.0,Gym / Fitness Center,Candy Store,Coffee Shop,American Restaurant,Japanese Restaurant,Grocery Store,Distribution Center,Discount Store,Diner,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Thai Restaurant,Sandwich Place,Bubble Tea Shop,Sushi Restaurant,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega,Wings Joint
2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1.0,Café,Coffee Shop,Pizza Place,Restaurant,Bakery,Plaza,Diner,Japanese Restaurant,Indian Restaurant,Beer Store
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1.0,Burger Joint,Coffee Shop,Dessert Shop,Wings Joint,Japanese Restaurant,Restaurant,Ramen Restaurant,Polish Restaurant,Park,Mexican Restaurant
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Spa,Food Truck,Mediterranean Restaurant,History Museum,Breakfast Spot,Gym / Fitness Center,Bakery,Coffee Shop,Furniture / Home Store,Distribution Center
6,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Café,Middle Eastern Restaurant,Burger Joint,Greek Restaurant,Bookstore,College Rec Center,Ramen Restaurant,Pub,Plaza
7,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1.0,Gastropub,Coffee Shop,Restaurant,Gym,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Poke Place,Camera Store,Italian Restaurant
8,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3.0,Restaurant,Breakfast Spot,Liquor Store,Wings Joint,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Distribution Center
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.0,Coffee Shop,Middle Eastern Restaurant,Café,Sandwich Place,Fried Chicken Joint,Spa,Pharmacy,Pizza Place,Deli / Bodega,Restaurant


In [79]:
# The code was removed by Watson Studio for sharing.

### Examine the 5 Clusters in Downtown Toronto

In [85]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park, Ontario Provincial Government",0.0,Thai Restaurant,Sandwich Place,Bubble Tea Shop,Sushi Restaurant,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega,Wings Joint


In [86]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"St. James Town, Cabbagetown",1.0,Café,Coffee Shop,Pizza Place,Restaurant,Bakery,Plaza,Diner,Japanese Restaurant,Indian Restaurant,Beer Store
4,Church and Wellesley,1.0,Burger Joint,Coffee Shop,Dessert Shop,Wings Joint,Japanese Restaurant,Restaurant,Ramen Restaurant,Polish Restaurant,Park,Mexican Restaurant
5,"Regent Park, Harbourfront",1.0,Spa,Food Truck,Mediterranean Restaurant,History Museum,Breakfast Spot,Gym / Fitness Center,Bakery,Coffee Shop,Furniture / Home Store,Distribution Center
6,"Garden District, Ryerson",1.0,Coffee Shop,Café,Middle Eastern Restaurant,Burger Joint,Greek Restaurant,Bookstore,College Rec Center,Ramen Restaurant,Pub,Plaza
7,St. James Town,1.0,Gastropub,Coffee Shop,Restaurant,Gym,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Poke Place,Camera Store,Italian Restaurant
9,Central Bay Street,1.0,Coffee Shop,Middle Eastern Restaurant,Café,Sandwich Place,Fried Chicken Joint,Spa,Pharmacy,Pizza Place,Deli / Bodega,Restaurant
10,"Richmond, Adelaide, King",1.0,Coffee Shop,Seafood Restaurant,Restaurant,Steakhouse,Asian Restaurant,Salad Place,Hotel,Concert Hall,Japanese Restaurant,Colombian Restaurant
11,"Harbourfront East, Union Station, Toronto Islands",1.0,Coffee Shop,Pizza Place,Sports Bar,Café,Theater,Plaza,Bubble Tea Shop,Sandwich Place,Salad Place,Gym
12,"Toronto Dominion Centre, Design Exchange",1.0,Coffee Shop,Bakery,Deli / Bodega,Restaurant,Café,Gym / Fitness Center,Gym,Japanese Restaurant,Shopping Mall,Seafood Restaurant
13,"Commerce Court, Victoria Hotel",1.0,Coffee Shop,Café,Deli / Bodega,Gastropub,Gym,American Restaurant,Japanese Restaurant,Hotel,Restaurant,Seafood Restaurant


In [87]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"CN Tower, King and Spadina, Railway Lands, Har...",2.0,Performing Arts Venue,Airport Lounge,Dance Studio,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Cupcake Shop


In [88]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Berczy Park,3.0,Restaurant,Breakfast Spot,Liquor Store,Wings Joint,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Distribution Center


In [89]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Christie,4.0,Gym / Fitness Center,Candy Store,Coffee Shop,American Restaurant,Japanese Restaurant,Grocery Store,Distribution Center,Discount Store,Diner,Dessert Shop
